In [1]:
import cv2
cv2.__version__

'4.5.4'

In [2]:
from PIL import Image

import napari
import scipy
import xlsxwriter
import imghdr
import colorsys

import numpy as np

import cv2 as cv
import matplotlib.pyplot as plt

import pyvista as pv

from csbdeep.utils import normalize
from scipy import ndimage as ndi

from skimage import filters
from skimage.draw import disk
# from skimage.segmentation import watershed
# from skimage.feature import peak_local_max
from vispy.color import Colormap
from IPython.display import display_html

from stardist.models import StarDist2D, Config3D, StarDist3D
from stardist.data import test_image_nuclei_2d
from stardist.plot import render_label

import sklearn
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM

from napari.settings import get_settings
settings = get_settings()

from aicsimageio import AICSImage

2024-12-17 12:55:46.385624: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-17 12:55:46.386261: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-17 12:55:46.389761: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-17 12:55:46.428853: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-17 12:55:47.193912: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [3]:
def ellipse_pixels(center, semi_axes, rotation, image_shape):
    y, x = np.meshgrid(np.arange(imarray.shape[1]), np.arange(imarray.shape[0]), indexing='xy')
    cos_theta = np.cos(rotation)
    sin_theta = np.sin(rotation)

    # Transform pixel coordinates to ellipse coordinates
    x_rot = cos_theta * (x - center[0]) + sin_theta * (y - center[1])
    y_rot = -sin_theta * (x - center[0]) + cos_theta * (y - center[1])

    # Ellipse equation: (x_rot / a)^2 + (y_rot / b)^2 <= 1
    mask = (x_rot / semi_axes[0])**2 + (y_rot / semi_axes[1])**2 <= 1

    settings.application.ipy_interactive = True

    # viewer_c = napari.Viewer()

    # viewer_c.add_image(y, name='Original', 
    #             colormap='grey', blending='additive')

    # Get pixel coordinates inside the ellipse
    return np.column_stack(np.where(mask))

def rgb_to_hsv(rgb_array):
    hsv_list = []
    for rgb in rgb_array:
        r, g, b = rgb  # Extract the components
        h, s, v = colorsys.rgb_to_hsv(r, g, b)  # Convert to HSV
        hsv_list.append([h * 360, s * 100, v * 100])  # Scale H to degrees, S and V to percentages
    return np.array(hsv_list)

## File upload
Write the name of the file that needs to be investigated.

In [4]:
tiff_file='cow1_d0_1_1.tif'

img=Image.open(tiff_file)
imarray=np.array(img)
print(imarray.shape)

(3056, 2920, 3)


In [5]:
r_X=1.0
r_Y=1.0

## ROI 
This section will find the region of interest by checking the blue part. Variables: [step] will dictate the size of the square of investigation (smaller values will get a better resolution but r 

In [6]:
ROI_image=imarray.copy()
mask_image=np.zeros(np.shape(imarray[:,:,1]))
step=5

for i in range(1+step,imarray.shape[0],step):
    for j in range(1+step,imarray.shape[1],step):
        #print(j)
        if(np.mean(ROI_image[i-step:i+step,j-step:j+step,2])<3+np.mean(ROI_image[i-step:i+step,j-step:j+step,1])):
            ROI_image[i:i+step,j:j+step]=0
        else:
            mask_image[i:i+step,j:j+step]=1

### Training image
Choose the image that will be used as training. It will be opened in Napari. Choose the circular selection on the right and choose as many nuclei as possible to train the algorithm. Then close the Napari window to continue

In [7]:
tiff_file='cow1_cont1a.tif'
image=Image.open(tiff_file)
imarray=np.array(image)

yes_points=[]

ROI_labels=np.zeros(imarray.shape)

settings.application.ipy_interactive = False

viewer_c = napari.Viewer()

viewer_c.add_image(imarray, name='Original', 
            colormap='grey', blending='additive')

def on_shape_added(layer, event):
    shapes_layer= event.source

shapes_layer = viewer_c.add_shapes(shape_type="circle", name="Nuclei training")

# Connect the click event to our callback function
viewer_c.mouse_drag_callbacks.append(on_shape_added)

napari.run()

for s in range(0,np.shape(shapes_layer.data)[0]):
    xmax=np.max(shapes_layer.data[s][:,0])
    ymax=np.max(shapes_layer.data[s][:,1])
    xmin=np.min(shapes_layer.data[s][:,0])
    ymin=np.min(shapes_layer.data[s][:,1])
    #print(shapes_layer.data)

    # Calculate ellipse parameters
    center = ((xmax + xmin) / 2, (ymax + ymin) / 2)
    semi_axes = (
        abs(xmax - xmin) / 2,  # Semi-major axis (height / 2)
        abs(ymax - ymin) / 2,  # Semi-minor axis (width / 2)
    )
    rotation = 0  # Napari's ellipses are axis-aligned by default

    image_shape=imarray.shape
    #print(center)
    #print(semi_axes)
    #print(image_shape)

    # Get the pixel coordinates inside the ellipse
    enclosed_pixels = ellipse_pixels(center, semi_axes, rotation, image_shape[:2])

    #print(enclosed_pixels)

    for i in range(enclosed_pixels.shape[0]):
        x=enclosed_pixels[i][0]
        y=enclosed_pixels[i][1]
        #print(new_point)
        yes_points.append([imarray[x,y,0],imarray[x,y,1],imarray[x,y,2]])

yes_points=np.array(yes_points)

libGL error: glx: failed to create dri3 screen
libGL error: failed to load driver: nouveau


After you run the next section, a new Napari window will appear with highlighted in green all the nuclei in the ROI.

In [ ]:
settings.application.ipy_interactive = True

imarray0=ROI_image.copy()

imR=imarray0[:,:,0]
imG=imarray0[:,:,1]
imB=imarray0[:,:,2]

yes_points_hsv=rgb_to_hsv(yes_points)

scaler = StandardScaler()
yes_points_scaled = scaler.fit_transform(yes_points_hsv)

# Train a one-class SVM
# `nu` controls the sensitivity (fraction of outliers allowed)
sensitivity = 0.1  # Choose sensitivity level (e.g., 10% of points allowed as outliers)
clf = OneClassSVM(kernel='rbf', nu=sensitivity, gamma=0.1) #'scale')
clf.fit(yes_points_scaled)

# Train SVM with polynomial kernel
# clf = SVC(kernel='poly', degree=5, C=1)
# clf.fit(X_scaled, y)

imV=np.zeros(np.shape(imR))
for i in range(step,np.shape(imV)[0]):
    for j in range(step,np.shape(imV)[1]):
        #print(pR)
        if (mask_image[i,j]>0):
            pR=imR[i,j]
            pG=imG[i,j]
            pB=imB[i,j]
            X = np.array([
                [pR, pG, pB]
            ])
            #print(X)
            X_hsv = rgb_to_hsv(X)
            X_scaled = scaler.transform(X_hsv)
            #print(X_scaled)
            predictions = clf.predict(X_scaled)
            #print(predictions)
            if predictions>0:
                imV[i,j]=1
                #yes_points=np.append(yes_points,X,axis=0)
            else:
                imV[i,j]=0
                #no_points=np.append(no_points,X,axis=0)
        else:
            j+=step

viewer_0 = napari.Viewer()

viewer_0.add_image(imarray0, name='Original', 
                colormap='grey', blending='additive')
viewer_0.add_image(imV, name='Violet', 
                colormap='green', blending='additive')